## Perceptron - versão inicial

Primeiramente fiz um perceptron com uma camada escondida e uma função sigmoide de ativação, ainda sem a utilização da função de custo e do treinamento para ver como o perceptron agia. Ao testar diferentes inputs, percebi que nenhuma predição chegava perto o suficiente para ser 1, sendo uma versão que não atende a porta XOR.

Isso aconteceu porque por não possuir o treinamento, não houve ajuste dos pesos, o que faz com que as saídas não sejam ajustadas conforme o erro.

In [33]:
import numpy as np

class Perceptron:
    def __init__(self, input_size, hidden_size, bias=1):
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        self.weights_hidden_output = np.random.rand(hidden_size, 1)
        self.bias_hidden = np.random.rand(hidden_size)
        self.bias_output = np.random.rand(1)

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward_pass(self, data):
        # Passagem pela camada escondida
        self.hidden_input = np.dot(data, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self._sigmoid(self.hidden_input)
        
        # Passagem pela camada de saída
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = self._sigmoid(self.final_input)
        
        return self.final_output

    def predict(self, data):
        output = self.forward_pass(data)
        return output, 1 if output > 0.9 else 0

# Exemplo de uso
input_data = np.array([0, 1])
perceptron = Perceptron(input_size=2, hidden_size=2)
output, prediction = perceptron.predict(input_data)
print(f"Input: {input_data}, Prediçãp: {output}, Retorno: {prediction}")


Input: [0 1], Prediçãp: [0.82908799], Retorno: 0


## Adicionando Função de custo

Aqui adicionei uma função de custo usando a média do erro quadrático, o que melhorou um pouco os números mais inda não foi suficiente para que as predições fossem corretas para uma porta XOR.

Isso acontece porque, apesar do erro estar sendo calculado, esse valor ainda não está sendo utilizado para mudar os pesos, o que significa que os resultados dessa versão são muito parecidos com o da anterior.

In [34]:
import numpy as np

class Perceptron:
    def __init__(self, input_size, hidden_size, bias=1):
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        self.weights_hidden_output = np.random.rand(hidden_size, 1)
        self.bias_hidden = np.random.rand(hidden_size)
        self.bias_output = np.random.rand(1)

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _mean_squared_error(self, predicted, actual):
        return np.mean((predicted - actual) ** 2)

    def forward_pass(self, data):
        # Passagem pela camada escondida
        self.hidden_input = np.dot(data, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self._sigmoid(self.hidden_input)
        
        # Passagem pela camada de saída
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = self._sigmoid(self.final_input)
        
        return self.final_output

    def predict(self, data):
        output = self.forward_pass(data)
        return output, 1 if output > 0.9 else 0

# Exemplo de uso
input_data = np.array([0, 1])
actual_output = 1
perceptron = Perceptron(input_size=2, hidden_size=2)
output, prediction = perceptron.predict(input_data)
error = perceptron._mean_squared_error(output, actual_output)
print(f"Input: {input_data}, Predição: {output}, Retorno: {prediction}, Erro: {error}")


Input: [0 1], Predição: [0.85289369], Retorno: 0, Erro: 0.021640266170998238


## Implementando o processo de treinamento

Em conclusão com o processo de treinamento,e ajuste dos erros, o Perceptron começou a de fato cumprir seu papel. Abaixo, é possível identificar que os valores aproximados condizem com a porta XOR, e apesar de nenhuma predição chegar perto de 1, é possível inferir que se for maior que 0.9, provavelmente a saída correta é 1. 
Por esse motivo, o print de saída mostra o input, o valor que foi estimado pelo Perceptron e também o retorno final que ele daria (definindo entre 0 e 1).



In [13]:
import numpy as np

class Perceptron:
    def __init__(self, input_size, hidden_size, learning_rate=0.1, bias=1):
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        self.weights_hidden_output = np.random.rand(hidden_size, 1)
        self.bias_hidden = np.random.rand(hidden_size)
        self.bias_output = np.random.rand(1)
        self.learning_rate = learning_rate

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _sigmoid_derivative(self, x):
        return x * (1 - x)

    def _mean_squared_error(self, predicted, actual):
        return np.mean((predicted - actual) ** 2)

    def forward_pass(self, data):
        # Passagem pela camada escondida
        self.hidden_input = np.dot(data, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self._sigmoid(self.hidden_input)
        
        # Passagem pela camada de saída
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = self._sigmoid(self.final_input)
        
        return self.final_output

    def predict(self, data):
        output = self.forward_pass(data)
        return output, 1 if output > 0.9 else 0

    def train(self, training_inputs, training_outputs, epochs):
        for epoch in range(epochs):
            for inputs, actual_output in zip(training_inputs, training_outputs):
                # Forward pass
                output = self.forward_pass(inputs)
                
                # Calcular erro na camada de saída
                output_error = actual_output - output
                output_delta = output_error * self._sigmoid_derivative(output)
                
                # Calcular erro na camada escondida
                hidden_error = output_delta.dot(self.weights_hidden_output.T)
                hidden_delta = hidden_error * self._sigmoid_derivative(self.hidden_output)
                
                # Atualizar pesos para camada escondida para camada de saída
                self.weights_hidden_output += self.hidden_output.reshape(-1, 1).dot(output_delta.reshape(1, -1)) * self.learning_rate
                self.bias_output += output_delta * self.learning_rate
                
                # Atualizar pesos para camada de entrada para camada escondida
                self.weights_input_hidden += inputs.reshape(-1, 1).dot(hidden_delta.reshape(1, -1)) * self.learning_rate
                self.bias_hidden += hidden_delta * self.learning_rate

# Dados de treinamento XOR
training_inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
training_outputs = np.array([0, 1, 1, 0])

# Treinando o Perceptron
perceptron = Perceptron(input_size=2, hidden_size=2)
perceptron.train(training_inputs, training_outputs, epochs=10000)

# Testando o Perceptron
for inputs in training_inputs:
    output, prediction = perceptron.predict(inputs)
    print(f"Input: {inputs}, Predict: {output}, Return: {prediction}")


Input: [0 0], Predict: [0.0639244], Return: 0
Input: [0 1], Predict: [0.94039637], Return: 1
Input: [1 0], Predict: [0.94031645], Return: 1
Input: [1 1], Predict: [0.0646812], Return: 0


## Backpropagation

Na versão acima, foi utilizado um ajuste simples de pesos, mas como a ponderada pede explicitamente um *backpropagation*, assim eu o farei.

O Backpropagation envolve o foward pass (que é calcular uma saída dada uma entrada, o que já está sendo utilizado), o backward pass (que calcula os gradientes do erro) e o ajuste dos pesos.

Abaixo, é possível visualizar nosso querido Perceptron com essa pequena atualização:

In [35]:
import numpy as np

class Perceptron:
    def __init__(self, input_size, hidden_size, learning_rate=0.1):
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        self.weights_hidden_output = np.random.rand(hidden_size, 1)
        self.bias_hidden = np.random.rand(hidden_size)
        self.bias_output = np.random.rand(1)
        self.learning_rate = learning_rate

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _sigmoid_derivative(self, x):
        return x * (1 - x)

    def _mean_squared_error(self, predicted, actual):
        return np.mean((predicted - actual) ** 2)

    def forward_pass(self, data):
        # Passagem pela camada escondida
        self.hidden_input = np.dot(data, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self._sigmoid(self.hidden_input)
        
        # Passagem pela camada de saída
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = self._sigmoid(self.final_input)
        
        return self.final_output

    def predict(self, data):
        output = self.forward_pass(data)
        return output, 1 if output > 0.9 else 0

    def train(self, training_inputs, training_outputs, epochs):
        for epoch in range(epochs):
            for inputs, actual_output in zip(training_inputs, training_outputs):
                # Forward pass
                output = self.forward_pass(inputs)
                
                # Backward pass
                # Calcular erro na camada de saída
                output_error = actual_output - output
                output_delta = output_error * self._sigmoid_derivative(output)
                
                # Calcular erro na camada escondida
                hidden_error = output_delta.dot(self.weights_hidden_output.T)
                hidden_delta = hidden_error * self._sigmoid_derivative(self.hidden_output)
                
                # Atualizar pesos para camada escondida para camada de saída
                self.weights_hidden_output += self.hidden_output.reshape(-1, 1).dot(output_delta.reshape(1, -1)) * self.learning_rate
                self.bias_output += output_delta * self.learning_rate
                
                # Atualizar pesos para camada de entrada para camada escondida
                self.weights_input_hidden += inputs.reshape(-1, 1).dot(hidden_delta.reshape(1, -1)) * self.learning_rate
                self.bias_hidden += hidden_delta * self.learning_rate

# Dados de treinamento XOR
training_inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
training_outputs = np.array([0, 1, 1, 0])

# Treinando o Perceptron
perceptron = Perceptron(input_size=2, hidden_size=2)
perceptron.train(training_inputs, training_outputs, epochs=10000)

# Testando o Perceptron
for inputs in training_inputs:
    output, prediction = perceptron.predict(inputs)
    print(f"Input: {inputs}, Predict: {output}, Return: {prediction}")


Input: [0 0], Predict: [0.06325786], Return: 0
Input: [0 1], Predict: [0.94023273], Return: 1
Input: [1 0], Predict: [0.94014328], Return: 1
Input: [1 1], Predict: [0.0653762], Return: 0


## Utilizando o Pytorch

Aqui eu utilizei o pytorch para fazer o perceptron, que é uma biblioteca que facilita o processo.
